In [1]:
import os
import pickle

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import sklearn.utils

from tqdm import tqdm

from model import FiringRateModel, PolynomialActivation
from train import train_model
from data import get_data, get_train_test_data
from evaluate import explained_variance_ratio
from utils import plot_predictions, plot_kernel
from config import config

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print(device)

cpu


In [3]:
def train(
    Is, 
    fs, 
    g,
    ds,
    cell_id, 
    bin_size,  
    device = None,
    hparams=[{"lr": 0.03, "gamma": 0.85, "step_size": 5, "epochs": 100}]
):
    best_model, best_losses = None, [0, 1e10]
    
    for i, hs in enumerate(hparams):
        print(f"Run {i}: {hs}")
        model = FiringRateModel(
            g.to(device), ds, bin_size=bin_size, device=device
        ).to(device)

        criterion = torch.nn.PoissonNLLLoss(log_input=False, reduction="none")
        optimizer = torch.optim.RMSprop(model.parameters(), lr=hs["lr"], centered=True)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=hs["gamma"], step_size=hs["step_size"])

        losses = train_model(
            model, 
            criterion, 
            optimizer,
            Is,
            fs,
            epochs = hs["epochs"],
            print_every = 1,
            bin_size = bin_size,
            up_factor = 1,
            scheduler = scheduler
        )
        
        if best_losses[-1] > losses[-1]:
            best_losses = losses
            best_model = model
    
    return best_model, best_losses

In [4]:
d = {}

bin_size = 20
ds = np.linspace(0.05, 1.0, 20)
actv_bin_size = 20

hparams = [
    {"lr": 0.03, "gamma": 0.85, "step_size": 5, "epochs": 150},
    {"lr": 0.02, "gamma": 0.90, "step_size": 10, "epochs": 200},
    {"lr": 0.01, "gamma": 0.95, "step_size": 20, "epochs": 250}
]

for cell_id in [476129135]:
    print(cell_id)
    data = get_data(cell_id, aligned=True)
    Is_tr, fs_tr, Is_te, fs_te, stims = get_train_test_data(data, bin_size, device=device)
    Is_tr, fs_tr, stims = sklearn.utils.shuffle(Is_tr, fs_tr, stims)

    actv = PolynomialActivation()
    actv.init_from_file(f"model/activation/poisson/bin_size_{actv_bin_size}/{cell_id}_0.pickle")
    model, losses = train(Is_tr, fs_tr, actv, ds, cell_id, bin_size, device=device, hparams=hparams)
    
    d[cell_id] = (model, losses)
    
    save = False
    plt.xlabel("epoch")
    plt.ylabel("loss")
    if save:
        plt.savefig(config["fig_save_path"] + f"{cell_id}/bin_size_{bin_size}/Loss_{actv_bin_size}.png")
        plt.close()

    for Is, fs, s in zip(Is_tr, fs_tr, stims):
        for i in range(Is.shape[0]):
            plot_predictions(
                model, 
                Is[i, :], 
                fs[i, :], 
                cell_id, 
                bin_size, 
                evr = None,
                save = save,
                fname = f"{s} {i}_{actv_bin_size}"
            )

    r = explained_variance_ratio(model, Is_te[0], fs_te[0], bin_size)
    rq = explained_variance_ratio(model, Is_te[0], fs_te[0], bin_size, quantize=True)
    plot_predictions(
        model, 
        Is_te[0][0, :], 
        fs_te[0][0, :], 
        cell_id, 
        bin_size, 
        evr = (r, rq),
        save = save,
        fname = f"Noise 2_{actv_bin_size}"
    )

    plot_kernel(
        model,
        cell_id,
        bin_size,
        save = save,
        fname = f"Kernel_{actv_bin_size}"
    )

476129135
Error


UnboundLocalError: local variable 'params' referenced before assignment

In [ ]:
params = {}
for fname in os.listdir("model/params"):
    with open(f"model/params/{fname}", "rb") as f:
        p = pickle.load(f)
        params[int(fname.split(".")[0])] = p

In [ ]:
zeros = 0
low_evr = []
for cell_id in params:
    evr = params[cell_id]["evr"]
    losses = params[cell_id]["losses"]
    print(cell_id, evr, losses[-1], len(losses))
    if evr == 0:
        zeros += 1
    if evr < 0.4 or losses[-1] > 1000:
        low_evr.append(cell_id)
print(zeros, len(params), zeros / len(params))

In [ ]:
print(len(low_evr))
with open("misc/cell_ids_rerun.csv", "w") as f:
    f.write(",".join(map(str, low_evr)))

In [ ]:
with open("model/max_firing_rates.pickle", "rb") as f:
    a = pickle.load(f)
    
#with open("misc/cell_ids_mfr.csv", "w") as f:
#    f.write(",".join(map(str, a.keys())))

In [ ]:
len(a.keys())